In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from scipy.stats import norm #用于从正态分布中采样随机数。这里正态分布代表每一帧输入的证据波动。
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 固定a，z设置为a/2
# 只考虑了match条件下的
# 定义条件对应的参数
conditions = {
    0: 'stranger',
    1: 'self'
}

# 定义参数
alaph1 = 1.5   # 自我, 1.8有点太离谱
alaph2 = -0.4   # 陌生人

beta1 = 0.2
beta2 = 0

gamma = 0.2

t0 = 0.2

# 定义 k(P) 随 P 变化的函数
def k_P(P, k_min = 0.01, k_max = 0.15, gamma = 0.1, P0 = 32):
    # 使用 sigmoid 函数平滑地调整 k 值
    return k_min + (k_max - k_min) / (1 + np.exp(-gamma * (P - P0)))

# 定义 sigmoid 函数 f(P)
def v_P_Function(P, P1 = 4, k_min = 0.01, k_max = 0.15, gamma = 0.1, P0 = 32):
    k = k_P(P, k_min, k_max, gamma, P0)  # k 随 P 变化
    return 1 / (1 + np.exp(-k * (P - P1)))

# 计算漂移速率 v，使用归一化的 T 和 P，这里归一化的原因是T和P的尺度差很多，为了平衡两者的影响
def compute_v(T, P, conditions):
    """
    计算漂移速率 v 基于未归一化的 T 和 P
    self条件下v增大
    """
    T_0 = 100
    k_T = 0.01  # 第一个函数的陡峭度

    v_T = 1 / (1 + np.exp(-k_T * (T - T_0)))
    v_P = v_P_Function(P = P, P1 = 4, k_min = 0.1, k_max = 0.05, gamma = gamma, P0 = 32)
    v_0 = v_T * v_P * 3

    # 根据 M 和 conditions 类型调整 a，使得条件影响更大
    if conditions == 1:
        v_1 = v_0 * (1 + alaph1)   # 条件是 self 时，增强影响
    else:  # condition_type == 'stranger'
        v_1 = v_0 * (1 + alaph2)  # 条件是 stranger 时，影响较小
    
    v = np.random.normal(v_1, 1)  # 从正态分布中抽取 v

    return v

# 计算阈值 a(W)，使用未归一化的 W
def compute_a(M):
    """
    计算阈值 a 基于未归一化的 W + T
    """
    k = 0.01  # sigmoid的陡峭度
    M_0 = 600  # sigmoid函数的中心点，大约在600附近

    a_0 = 1 / (1 + np.exp(-k * (M - M_0))) * 3

    # 根据 M 和 conditions 类型调整 a，使得条件影响更大
    if M > 600:
            a_1 = a_0 * (1 + beta1)  # 在 M > 600 时，增强影响
    else:
            a_1 = a_0 * (1 + beta2)  # 在 M <= 600 时，影响较小

    a = np.random.normal(a_1, 0.5)  # 从正态分布中抽取 a

    return a

In [3]:
# 整体数据
# 生成数据：模拟证据积累过程
def generate_data(num_subjects, num_trials_per_subject):
    all_data = []  # 存储所有被试的数据
    
    # 对每个被试进行模拟
    for subject in range(num_subjects):
        trials_data = []  # 存储每个试验的 RT 和 response 数据
        
        # 为每个被试随机选择 P, T, W
        T = np.random.randint(10, 600)  # 随机选择时间 T
        P = np.random.randint(0, 150)  # 随机选择刺激强度 P
        W = np.random.randint(200, 1500)  # 随机选择反应窗口 W
        M = T + W

        # 在被试层面计算阈值 a，只计算一次，应用到所有试次
        a = compute_a(M)  # 只计算一次，应用到所有试次

        # 如果 a 小于 0，则重新抽取
        while a < 0:
            a = compute_a(M)  # 重新计算 a

        # 在每个被试中确保条件下试次数相等
        trials_per_condition = num_trials_per_subject // 2  # 每个条件的试次数
        
        # 如果是奇数试次，随机选择一个条件分配多一试次
        if num_trials_per_subject % 2 != 0:
            extra_trial_condition = np.random.choice([0, 1])
            trials_per_condition += 1

        # 为每个被试生成试验数据，分配不同条件
        for condition_key in range(2):  # 0: 'self', 1: 'stranger'
            condition = conditions[condition_key]  # 获取条件名

            # 初始化该条件下的试次计数
            trial_count = 0

            while trial_count < trials_per_condition:
                retry_count = 0  # 重试次数
                valid_trial = False  # 标记该试次是否有效

                # 如果试次不符合条件且重试次数未达到最大值
                if retry_count < 10 and not valid_trial:
                    retry_count += 1
        
                # 计算漂移速率 v，随试次变化
                v = compute_v(T, P, condition_key)  # 计算漂移速率 (使用归一化数据)

                # 初始化证据，设定初始证据为一半的边界值 (即 z = a / 2)
                evidence = a / 2
                delta_t = 0.001
                max_time = (W + T - t0) * 0.001 + 0.001 # 最大时间限制
                decision_time = 0  # 记录决策发生的时间

                # 循环模拟证据积累过程，直到证据达到阈值或最大时间
                time = 0  # 当前模拟时间
                while evidence > 0 and evidence < a:  # 直到证据达到 0 或 a

                    # 证据更新：漂移 + 噪声
                    instantaneous_evidence = v * delta_t  # 漂移项
                    noise = norm.rvs(loc=0, scale=1) * np.sqrt(delta_t)  # 从正态分布中采样噪声
                    evidence += instantaneous_evidence + noise  # 更新证据
                    time += delta_t  # 增加时间

                    # 判断是否在当前步骤达到阈值并根据证据值确定决策时间和反应类别
                    if evidence >= a:
                        decision_time = time + t0  # 证据达到上边界时的决策时间
                        response = 1  # 根据证据值确定反应类别（反应 1 为证据达到上边界）
                        break  # 如果达到上边界，退出循环
                    elif evidence <= 0:
                        decision_time = time + t0  # 证据达到下边界时的决策时间
                        response = 2  # 根据证据值确定反应类别（反应 2 为证据达到下边界）
                        break  # 如果达到下边界，退出循环

                # 过滤掉超出 (T + W) * 0.001 的反应时间
                if decision_time < max_time:
                    trials_data.append({
                        'subjectID': subject + 1,  # 被试编号
                        'trialID': trial_count + 1,  # 试验编号
                        'T': T,     # 刺激呈现时间
                        'P': P,     # 刺激强度
                        'W': W,     # 反应窗口
                        'Label': condition,  # 当前条件
                        'v': v,     # 漂移率
                        'a': a,     # 反应阈值
                        'RT': decision_time,   # 反应时间
                        'response': response  # 按键反应
                    })
                    trial_count += 1  # 有效试次数增加

        # 将所有试验数据添加到全局数据列表
        all_data.extend(trials_data)

    # 将所有试验数据转换为 DataFrame
    df = pd.DataFrame(all_data)
    return df

In [ ]:
# 10000个被试，260个trial要跑1201m
num_subjects = 10000  # 设定被试数量
num_trials_per_subject = 260  # 每个被试的试次数量
test = generate_data(num_subjects, num_trials_per_subject)  # 获取数据框

print(test)  # 打印数据框

# 可选择保存数据框到文件                                                                                                                       
test.to_csv("250430.csv", index=False)  # 保存为 CSV 文件  

         subjectID  trialID    T   P     W     Label         v        a  \
0                1        1  369   5   311  stranger  0.963123  2.94987   
1                1        2  369   5   311  stranger  2.671695  2.94987   
2                1        3  369   5   311  stranger  1.790672  2.94987   
3                1        4  369   5   311  stranger  2.009321  2.94987   
4                1        5  369   5   311  stranger  1.522518  2.94987   
...            ...      ...  ...  ..   ...       ...       ...      ...   
2599995      10000      126   18  52  1047      self  3.176697  3.25785   
2599996      10000      127   18  52  1047      self  3.842193  3.25785   
2599997      10000      128   18  52  1047      self  1.283697  3.25785   
2599998      10000      129   18  52  1047      self  2.251978  3.25785   
2599999      10000      130   18  52  1047      self  1.894840  3.25785   

            RT  response  
0        0.442         1  
1        0.403         1  
2        0.640    

In [3]:
# 条件数据
# 生成数据：模拟证据积累过程
def generate_data(num_subjects, num_trials_per_subject, P, T, W):
    all_data = []  # 存储所有被试的数据
    
    # 对每个被试进行模拟
    for subject in range(num_subjects):
        trials_data = []  # 存储每个试验的 RT 和 response 数据
        
        # 为每个被试随机选择 P, T, W
        P = P  # 随机选择刺激强度 P
        T = T  # 随机选择时间 T    
        W = W  # 随机选择反应窗口 W
        M = T + W

        # 在被试层面计算阈值 a，只计算一次，应用到所有试次
        a = compute_a(M)  # 只计算一次，应用到所有试次

        # 如果 a 小于 0，则重新抽取
        while a < 0:
            a = compute_a(M)  # 重新计算 a

        # 在每个被试中确保条件下试次数相等
        trials_per_condition = num_trials_per_subject // 2  # 每个条件的试次数
        
        # 如果是奇数试次，随机选择一个条件分配多一试次
        if num_trials_per_subject % 2 != 0:
            extra_trial_condition = np.random.choice([0, 1])
            trials_per_condition += 1

        # 为每个被试生成试验数据，分配不同条件
        for condition_key in range(2):  # 0: 'self', 1: 'stranger'
            condition = conditions[condition_key]  # 获取条件名

            # 初始化该条件下的试次计数
            trial_count = 0

            while trial_count < trials_per_condition:
                retry_count = 0  # 重试次数
                valid_trial = False  # 标记该试次是否有效

                # 如果试次不符合条件且重试次数未达到最大值
                if retry_count < 10 and not valid_trial:
                    retry_count += 1
        
                # 计算漂移速率 v，随试次变化
                v = compute_v(T, P, condition_key)  # 计算漂移速率 (使用归一化数据)

                # 初始化证据，设定初始证据为一半的边界值 (即 z = a / 2)
                evidence = a / 2
                delta_t = 0.001
                max_time = (W + T - t0) * 0.001 + 0.001 # 最大时间限制
                decision_time = 0  # 记录决策发生的时间

                # 循环模拟证据积累过程，直到证据达到阈值或最大时间
                time = 0  # 当前模拟时间
                while evidence > 0 and evidence < a:  # 直到证据达到 0 或 a

                    # 证据更新：漂移 + 噪声
                    instantaneous_evidence = v * delta_t  # 漂移项
                    noise = norm.rvs(loc=0, scale=1) * np.sqrt(delta_t)  # 从正态分布中采样噪声
                    evidence += instantaneous_evidence + noise  # 更新证据
                    time += delta_t  # 增加时间

                    # 判断是否在当前步骤达到阈值并根据证据值确定决策时间和反应类别
                    if evidence >= a:
                        decision_time = time + t0  # 证据达到上边界时的决策时间
                        response = 1  # 根据证据值确定反应类别（反应 1 为证据达到上边界）
                        break  # 如果达到上边界，退出循环
                    elif evidence <= 0:
                        decision_time = time + t0  # 证据达到下边界时的决策时间
                        response = 2  # 根据证据值确定反应类别（反应 2 为证据达到下边界）
                        break  # 如果达到下边界，退出循环

                # 过滤掉超出 (T + W) * 0.001 的反应时间
                if decision_time < max_time:
                    trials_data.append({
                        'subjectID': subject + 1,  # 被试编号
                        'trialID': trial_count + 1,  # 试验编号
                        'T': T,     # 刺激呈现时间
                        'P': P,     # 刺激强度
                        'W': W,     # 反应窗口
                        'Label': condition,  # 当前条件
                        'v': v,     # 漂移率
                        'a': a,     # 反应阈值
                        'RT': decision_time,   # 反应时间
                        'response': response  # 按键反应
                    })
                    trial_count += 1  # 有效试次数增加

        # 将所有试验数据添加到全局数据列表
        all_data.extend(trials_data)

    # 将所有试验数据转换为 DataFrame
    df = pd.DataFrame(all_data)
    return df

In [4]:
np.random.seed(39)
num_subjects = 8  # 设定被试数量
num_trials_per_subject = 260  # 每个被试的试次数量
con1 = generate_data(num_subjects, num_trials_per_subject, 0, 30, 300)  # 获取数据框

print(con1)  # 打印数据框

# 可选择保存数据框到文件                                                                                                                       
con1.to_csv("d1.csv", index=False)  # 保存为 CSV 文件   

      subjectID  trialID   T  P    W     Label         v         a     RT  \
0             1        1  30  0  300  stranger -0.574236  0.891340  0.254   
1             1        2  30  0  300  stranger -1.157028  0.891340  0.264   
2             1        3  30  0  300  stranger  0.354680  0.891340  0.307   
3             1        4  30  0  300  stranger -0.214056  0.891340  0.239   
4             1        5  30  0  300  stranger  0.971316  0.891340  0.281   
...         ...      ...  .. ..  ...       ...       ...       ...    ...   
2075          8      126  30  0  300      self  1.370738  0.568073  0.240   
2076          8      127  30  0  300      self -1.152322  0.568073  0.278   
2077          8      128  30  0  300      self  1.343913  0.568073  0.274   
2078          8      129  30  0  300      self  0.855316  0.568073  0.228   
2079          8      130  30  0  300      self  1.948119  0.568073  0.271   

      response  
0            1  
1            2  
2            1  
3      

In [5]:
np.random.seed(39)
num_subjects = 8  # 设定被试数量
num_trials_per_subject = 260  # 每个被试的试次数量
con2 = generate_data(num_subjects, num_trials_per_subject, 0, 30, 600)  # 获取数据框

print(con2)  # 打印数据框

# 可选择保存数据框到文件                                                                                                                       
con2.to_csv("d2.csv", index=False)  # 保存为 CSV 文件 

      subjectID  trialID   T  P    W     Label         v         a     RT  \
0             1        1  30  0  600  stranger  1.359820  2.770413  0.598   
1             1        2  30  0  600  stranger  2.165586  2.770413  0.588   
2             1        3  30  0  600  stranger  2.634394  2.770413  0.418   
3             1        4  30  0  600  stranger  0.567938  2.770413  0.403   
4             1        5  30  0  600  stranger  1.271102  2.770413  0.342   
...         ...      ...  .. ..  ...       ...       ...       ...    ...   
2075          8      126  30  0  600      self  2.521324  2.394477  0.440   
2076          8      127  30  0  600      self  2.732031  2.394477  0.481   
2077          8      128  30  0  600      self  3.696504  2.394477  0.471   
2078          8      129  30  0  600      self  2.837343  2.394477  0.562   
2079          8      130  30  0  600      self  0.497435  2.394477  0.399   

      response  
0            1  
1            1  
2            1  
3      

In [6]:
np.random.seed(39)
num_subjects = 8  # 设定被试数量
num_trials_per_subject = 260  # 每个被试的试次数量
con3 = generate_data(num_subjects, num_trials_per_subject, 120, 30, 600)  # 获取数据框

print(con3)  # 打印数据框

# 可选择保存数据框到文件                                                                                                                       
con3.to_csv("d3.csv", index=False)  # 保存为 CSV 文件 

      subjectID  trialID   T    P    W     Label         v         a     RT  \
0             1        1  30  120  600  stranger  1.340266  2.770413  0.589   
1             1        2  30  120  600  stranger  0.472344  2.770413  0.619   
2             1        3  30  120  600  stranger  1.491858  2.770413  0.583   
3             1        4  30  120  600  stranger  1.871066  2.770413  0.534   
4             1        5  30  120  600  stranger  2.990117  2.770413  0.412   
...         ...      ...  ..  ...  ...       ...       ...       ...    ...   
2075          8      126  30  120  600      self  2.557763  2.993353  0.489   
2076          8      127  30  120  600      self  2.642294  2.993353  0.587   
2077          8      128  30  120  600      self  1.867521  2.993353  0.376   
2078          8      129  30  120  600      self  3.828733  2.993353  0.620   
2079          8      130  30  120  600      self  3.980589  2.993353  0.504   

      response  
0            1  
1            1  


In [7]:
np.random.seed(39)
num_subjects = 8  # 设定被试数量
num_trials_per_subject = 260  # 每个被试的试次数量
con4 = generate_data(num_subjects, num_trials_per_subject, 120, 80, 600)  # 获取数据框

print(con4)  # 打印数据框

# 可选择保存数据框到文件                                                                                                                       
con4.to_csv("d4.csv", index=False)  # 保存为 CSV 文件 

      subjectID  trialID   T    P    W     Label         v         a     RT  \
0             1        1  80  120  600  stranger  2.485827  3.186328  0.608   
1             1        2  80  120  600  stranger  2.102450  3.186328  0.646   
2             1        3  80  120  600  stranger  1.788999  3.186328  0.659   
3             1        4  80  120  600  stranger  2.114606  3.186328  0.629   
4             1        5  80  120  600  stranger  3.890860  3.186328  0.460   
...         ...      ...  ..  ...  ...       ...       ...       ...    ...   
2075          8      126  80  120  600      self  4.385307  2.787420  0.375   
2076          8      127  80  120  600      self  3.019217  2.787420  0.410   
2077          8      128  80  120  600      self  3.264219  2.787420  0.623   
2078          8      129  80  120  600      self  2.001880  2.787420  0.625   
2079          8      130  80  120  600      self  3.273875  2.787420  0.608   

      response  
0            1  
1            1  


In [8]:
np.random.seed(39)
num_subjects = 8  # 设定被试数量
num_trials_per_subject = 260  # 每个被试的试次数量
con5 = generate_data(num_subjects, num_trials_per_subject, 8, 100, 1100)  # 获取数据框

print(con5)  # 打印数据框

# 可选择保存数据框到文件                                                                                                                       
con5.to_csv("d5.csv", index=False)  # 保存为 CSV 文件 

      subjectID  trialID    T  P     W     Label         v         a     RT  \
0             1        1  100  8  1100  stranger  1.457933  4.293518  0.901   
1             1        2  100  8  1100  stranger  2.192985  4.293518  0.805   
2             1        3  100  8  1100  stranger  2.717217  4.293518  0.947   
3             1        4  100  8  1100  stranger  0.851193  4.293518  0.811   
4             1        5  100  8  1100  stranger  0.763546  4.293518  1.178   
...         ...      ...  ... ..   ...       ...       ...       ...    ...   
2075          8      126  100  8  1100      self  2.156995  4.271080  1.149   
2076          8      127  100  8  1100      self  2.596501  4.271080  0.751   
2077          8      128  100  8  1100      self  1.623218  4.271080  1.086   
2078          8      129  100  8  1100      self  1.833504  4.271080  1.060   
2079          8      130  100  8  1100      self  1.614235  4.271080  1.150   

      response  
0            1  
1            1  


In [9]:
np.random.seed(39)
num_subjects = 8  # 设定被试数量
num_trials_per_subject = 260  # 每个被试的试次数量
con6 = generate_data(num_subjects, num_trials_per_subject, 120, 500, 1500)  # 获取数据框

print(con6)  # 打印数据框

# 可选择保存数据框到文件                                                                                                                       
con6.to_csv("d6.csv", index=False)  # 保存为 CSV 文件 

      subjectID  trialID    T    P     W     Label         v         a     RT  \
0             1        1  500  120  1500  stranger  1.983410  4.302417  1.359   
1             1        2  500  120  1500  stranger  1.026752  4.302417  1.242   
2             1        3  500  120  1500  stranger  2.257919  4.302417  1.786   
3             1        4  500  120  1500  stranger  1.521800  4.302417  1.089   
4             1        5  500  120  1500  stranger  1.422470  4.302417  1.077   
...         ...      ...  ...  ...   ...       ...       ...       ...    ...   
2075          8      126  500  120  1500      self  6.379804  3.733395  0.400   
2076          8      127  500  120  1500      self  6.210046  3.733395  0.528   
2077          8      128  500  120  1500      self  7.225616  3.733395  0.609   
2078          8      129  500  120  1500      self  8.032187  3.733395  0.554   
2079          8      130  500  120  1500      self  6.349857  3.733395  0.554   

      response  
0         

In [10]:
np.random.seed(45)
num_subjects = 8  # 设定被试数量
num_trials_per_subject = 260  # 每个被试的试次数量
con7 = generate_data(num_subjects, num_trials_per_subject, 120, 30, 800)  # 获取数据框

print(con7)  # 打印数据框

# 可选择保存数据框到文件                                                                                                                       
con7.to_csv("d7.csv", index=False)  # 保存为 CSV 文件 

      subjectID  trialID   T    P    W     Label         v         a     RT  \
0             1        1  30  120  800  stranger -1.557390  3.285145  0.662   
1             1        2  30  120  800  stranger  0.633250  3.285145  0.642   
2             1        3  30  120  800  stranger  2.012249  3.285145  0.782   
3             1        4  30  120  800  stranger -1.710996  3.285145  0.584   
4             1        5  30  120  800  stranger  1.655292  3.285145  0.598   
...         ...      ...  ..  ...  ...       ...       ...       ...    ...   
2075          8      126  30  120  800      self  2.312918  3.717869  0.597   
2076          8      127  30  120  800      self  2.737086  3.717869  0.757   
2077          8      128  30  120  800      self  1.195936  3.717869  0.631   
2078          8      129  30  120  800      self  3.985783  3.717869  0.498   
2079          8      130  30  120  800      self  0.712383  3.717869  0.610   

      response  
0            2  
1            1  


In [11]:
np.random.seed(45)
num_subjects = 8  # 设定被试数量
num_trials_per_subject = 260  # 每个被试的试次数量
con8 = generate_data(num_subjects, num_trials_per_subject, 120, 80, 800)  # 获取数据框

print(con8)  # 打印数据框

# 可选择保存数据框到文件                                                                                                                       
con8.to_csv("d8.csv", index=False)  # 保存为 CSV 文件 

      subjectID  trialID   T    P    W     Label         v         a     RT  \
0             1        1  80  120  800  stranger  2.377594  3.406820  0.863   
1             1        2  80  120  800  stranger -0.203875  3.406820  0.620   
2             1        3  80  120  800  stranger  0.427956  3.406820  0.675   
3             1        4  80  120  800  stranger -0.859840  3.406820  0.810   
4             1        5  80  120  800  stranger  1.509022  3.406820  0.615   
...         ...      ...  ..  ...  ...       ...       ...       ...    ...   
2075          8      126  80  120  800      self  3.919580  3.605505  0.440   
2076          8      127  80  120  800      self  3.240826  3.605505  0.634   
2077          8      128  80  120  800      self  2.418779  3.605505  0.876   
2078          8      129  80  120  800      self  3.488072  3.605505  0.630   
2079          8      130  80  120  800      self  2.954929  3.605505  0.861   

      response  
0            1  
1            1  
